In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

2023-09-18 18:29:40.488885: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 18:29:40.959708: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
batch_size = 2048
l_rate = 0.001

In [3]:
@tf.keras.saving.register_keras_serializable()
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense2 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense3 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense4 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense5 = tf.keras.layers.Dense(units=8)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        output = self.dense5(x)
        return output

In [4]:
def valiAll(index_epoch):
    y_v_p = model(X_v)
    va_mse = tf.reduce_mean(tf.square(y_v_p - y_v))
    va_rmse = tf.sqrt(va_mse)
    va_mae = tf.reduce_mean(tf.abs(y_v_p - y_v))
    va_r2 = 1 - tf.reduce_sum(tf.square(y_v_p - y_v)) / tf.reduce_sum(tf.square(y_v - tf.reduce_mean(y_v)))
    print("mse:{} rmse:{} mae:{} r2:{}".format(va_mse, va_rmse, va_mae, va_r2))

In [5]:
test_dataset = pd.read_csv("testset.csv", encoding='utf-8').sample(frac=1).reset_index(drop=True)
X_v = test_dataset.loc[:,'freq':'L2'].to_numpy(dtype = np.float32)
y_v = test_dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)

In [6]:
dataset1 = pd.read_csv('./20-24Trainset.csv', encoding='utf-8')
dataset2 = pd.read_csv('./50-54Trainset.csv', encoding='utf-8')
dataset = pd.concat([dataset1, dataset2], ignore_index=True).sample(frac=1).reset_index(drop=True)
# dataset = dataset[(dataset['freq'] >= 1.0) & (dataset['freq'] <= 2.6)]
X = dataset.loc[:,'freq':'L2'].to_numpy(dtype = np.float32)
y = dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)
dataset_train = tf.data.Dataset.from_tensor_slices((X, y))
dataset_train = dataset_train.shuffle(buffer_size=X.shape[0])
dataset_train = dataset_train.batch(batch_size)
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

2023-09-18 18:29:42.813437: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-09-18 18:29:42.813889: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [7]:
model = MLP()
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate)

In [8]:
for i in range(150):
    for X, y in dataset_train:
        with tf.GradientTape() as tape:
            y_pred = model(X)
            tr_mse = tf.reduce_mean(tf.square(y_pred - y))
        tr_rmse = tf.sqrt(tr_mse)
        tr_mae = tf.reduce_mean(tf.abs(y_pred - y))
        tr_r2 = 1 - tf.reduce_sum(tf.square(y_pred - y)) / tf.reduce_sum(tf.square(y - tf.reduce_mean(y)))
        grads = tape.gradient(tr_mse, model.variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
    print("epoch:{}".format(i))
    print("train mse:{} rmse:{} mae:{} r2:{}".format(tr_mse, tr_rmse, tr_mae, tr_r2))
    valiAll(i)

2023-09-18 18:29:43.842354: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x562ad64b47a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-18 18:29:43.842380: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2023-09-18 18:29:43.842384: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2023-09-18 18:29:43.845775: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-18 18:29:43.949538: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-18 18:29:44.058727: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


epoch:0
train mse:0.05951593443751335 rmse:0.24395887553691864 mae:0.19133226573467255 r2:0.512557864189148
mse:0.060445789247751236 rmse:0.24585725367069244 mae:0.19305388629436493 r2:0.5049315690994263
epoch:1
train mse:0.05100033059716225 rmse:0.2258325219154358 mae:0.1750820428133011 r2:0.5823920965194702
mse:0.05047406628727913 rmse:0.2246643453836441 mae:0.17570842802524567 r2:0.5866028070449829
epoch:2
train mse:0.047532662749290466 rmse:0.21801985800266266 mae:0.17075784504413605 r2:0.6105225086212158
mse:0.0457192063331604 rmse:0.21382050216197968 mae:0.1662796437740326 r2:0.6255465745925903
epoch:3
train mse:0.041642460972070694 rmse:0.20406484603881836 mae:0.15785786509513855 r2:0.6589142084121704
mse:0.041213568300008774 rmse:0.20301124453544617 mae:0.15768875181674957 r2:0.6624490022659302
epoch:4
train mse:0.03894885629415512 rmse:0.19735464453697205 mae:0.1525951623916626 r2:0.6812013387680054
mse:0.03833235427737236 rmse:0.1957864910364151 mae:0.15069057047367096 r2:0.6